In [11]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


In [12]:
import warnings
warnings.filterwarnings('ignore')

import torch, os
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, models

In [13]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomRotation(20),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
}


In [14]:
data_dir = '/kaggle/input/fundus-pytorch'
image_datasets = {x: datasets.ImageFolder(
    root=f"{data_dir}/{x}",
    transform=data_transforms[x]
) for x in ['train', 'test', 'val']}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True) for x in ['train', 'test', 'val']}

In [15]:
class GlaucomaCNN(nn.Module):
    def __init__(self):
        super(GlaucomaCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 32 * 32, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 2)
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GlaucomaCNN()

if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = nn.DataParallel(model)

model = model.to(device)
print(device)

Using 2 GPUs!
cuda


In [17]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Training loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print("-" * 30)
        
        for batch_idx, (inputs, labels) in enumerate(dataloaders['train']):
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            if (batch_idx + 1) % 10 == 0:
                print(f"Batch {batch_idx+1}/{len(dataloaders['train'])} | Loss: {loss.item():.4f}")
        
        epoch_loss = running_loss / len(dataloaders['train'])
        epoch_acc = 100. * correct / total
        print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%\n")
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss /= len(dataloaders['val'])
        val_acc = 100. * val_correct / val_total
        print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_acc:.2f}%\n")

train_model(model, dataloaders, criterion, optimizer, num_epochs=50)

Epoch 1/50
------------------------------
Batch 10/270 | Loss: 0.6362
Batch 20/270 | Loss: 0.6852
Batch 30/270 | Loss: 0.6668
Batch 40/270 | Loss: 0.5900
Batch 50/270 | Loss: 0.6612
Batch 60/270 | Loss: 0.6277
Batch 70/270 | Loss: 0.6657
Batch 80/270 | Loss: 0.7120
Batch 90/270 | Loss: 0.6441
Batch 100/270 | Loss: 0.5697
Batch 110/270 | Loss: 0.6047
Batch 120/270 | Loss: 0.6178
Batch 130/270 | Loss: 0.6786
Batch 140/270 | Loss: 0.6799
Batch 150/270 | Loss: 0.6373
Batch 160/270 | Loss: 0.6271
Batch 170/270 | Loss: 0.6065
Batch 180/270 | Loss: 0.6601
Batch 190/270 | Loss: 0.5628
Batch 200/270 | Loss: 0.5826
Batch 210/270 | Loss: 0.5817
Batch 220/270 | Loss: 0.6135
Batch 230/270 | Loss: 0.7217
Batch 240/270 | Loss: 0.6900
Batch 250/270 | Loss: 0.5840
Batch 260/270 | Loss: 0.3508
Batch 270/270 | Loss: 0.6153
Epoch 1 Loss: 0.6314 | Accuracy: 67.96%

Validation Loss: 0.5816 | Validation Accuracy: 71.32%

Epoch 2/50
------------------------------
Batch 10/270 | Loss: 0.5257
Batch 20/270 | Los

In [19]:
def evaluate_model(model, dataloader, criterion):
    model.eval()
    total = 0
    correct = 0
    running_loss = 0.0
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    test_loss = running_loss / len(dataloader)
    test_acc = 100. * correct / total
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

evaluate_model(model, dataloaders['test'], criterion)

Test Loss: 0.2220 | Test Accuracy: 90.47%


In [20]:
torch.save(model.state_dict(), "glaucoma_model_90TA.pth")
print("Model saved successfully!")

Model saved successfully!
